# Реализация нахождения уравнений движения с помощью Урванений Татаринова 

##### TODO 
 - Добавить рисунок и описание решаемой здесь задачи
 - Добавить решение с помощью общих теорем динамики
 - Тесты

In [32]:
import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Eq, Idx, Sum, Function, Matrix
from sympy import diff, symbols, solve, simplify, poly, pprint
from sympy import cos, sin

from abc import ABC, abstractmethod

### Уравнения Татаринова
$$
\frac{d}{dt} \frac{\partial L*}{\partial \omega_{\alpha}} +
\{P_{\alpha}, L*\} = 
\{P_{\alpha}, \sum_{\mu}{\omega_{\mu} P_{\mu}} \} +
\sum_{i}{Q_i \frac{\partial v_i}{ \partial \omega_{\alpha}}}
$$

### Скобка Пуассона
$$
\{f, g\} = \sum_{i=1}^{N} \left( \frac{\partial f}{\partial q_{i}} \frac{\partial g}{\partial p_{i}} - \frac{\partial f}{\partial p_i} \frac{\partial g}{\partial q_i}\right).
$$

### Функции $ P_{\alpha} $
$$
\sum_{\alpha}{P_{\alpha} \omega_{\alpha}} = \sum_i{p_iv_i}
$$

### Функции $v_i$
$$
v_i = \dot{q}
$$

In [33]:
class MechanicalSystem():
    
    def __init__(self):
        pass
    
    def set_q(self, q):
        self.q = q
    
    def set_p(self, p):
        self.p = p
    
    def poisson_bracket(self, F, G): # надо сделать, чтобы использовалось IndexedBase
        """ To evaluate an unevaluated derivative, use the doit method. """
        res = 0
        for i in range(3):
            res += diff(F, self.q[i])*diff(G, self.p[i])
            res -= diff(F, self.p[i])*diff(G, self.q[i])
        return res
    
    @staticmethod
    def sub_Eq(equation, Eq):
        return equation.subs(Eq.args[0], Eq.args[1])
    
    @classmethod
    def sub_Eqs(cls, equation, Eqs):
        sub_equation = equation
        for Eq in Eqs:
            sub_equation = cls.sub_Eq(sub_equation, Eq)
        return sub_equation 
    
    @staticmethod
    def left_part_Eqs(Eqs):
        return [Eq.args[0] for Eq in Eqs]
    
    @staticmethod
    def right_part_Eqs(Eqs):
        return [Eq.args[1] for Eq in Eqs]
    
    def set_connections(self, connections):
        """ Setting up connections """
        self.connections = connections
        
    def sub_connections(self, func):
        """ Substitutes connections in the equation
        Private special case:
            L -> L* 
        """
        sub_dict = {conn.args[0]: conn.args[1] for conn in self.connections}
        return simplify(func.subs(sub_dict))
    
    def sub_connections_to_list(self, equations):
        return [self.sub_connections(equation) for equation in equations]
    
    @staticmethod
    def diff_hack(equation, by):
        """ eq -> eq.subs(Derivative -> temp_variable) -> eq.diff(temp_variable) ->
            -> eq.subs(temp_variable -> Derivative)
        """
        tmp_by = Symbol('tmp_by')
        tmp_eq = equation.subs({by: tmp_by})
        tmp_eq = tmp_eq.diff(tmp_by)
        return tmp_eq.subs({tmp_by: by})
        

In [34]:
class TatarinovSystem(MechanicalSystem):
    
    def __init__(self, N=3):
        super().__init__()
        self.N = N
        self.tatarinov_equations = [None]*N
        
    def set_omega_equations(self, omegas, equations):
        omega_equations = [Eq(omega, equation) for omega, equation in zip(omegas, equations)]
        self.omega_equations = omega_equations
        
    def set_v_equations(self, vs, equations): # vs - не лучшее название
        v_equations = [Eq(v, equation) for v, equation in zip(vs, equations)]
        self.v_equations = v_equations
    
    def set_v(self):
        v = IndexedBase('v')
        k= symbols('k', cls=Idx)
        Eq(v[k], diff(self.q[k]))
        
    def set_P(self):
        k, mu = symbols('k, mu', cls=Idx)
        left = Sum(P[k]*om[k], (k, 1, self.N)).doit()
        right = Sum(p[i]*v[i], (k, 1, self.N))
        Eq(left, right)
    
    def set_L(self, L):
        self.L = L
    
    def set_F(self, F):
        self.F = F
    
    def create_r(self):
        r = {}
        r['s'] = x*e['x'] + y*e['y']
        r['p'] = r['s'] + xi*e['xi'] + eta*e['eta']
        self.r = r
        return self.r

    def set_Qs(self, Qs):
        self.Qs = Qs
    
    def create_P(self):
        equations_for_P = Eq(
            Sum(P[k]*omega[k], (k, 1, S.N)),
            Sum(p[k]*v[k], (k, 1, S.N)))
        print(equations_for_P)
        # subs omega_i and v_i
        sub_equations_for_P = TatarinovSystem.sub_Eqs(
            equations_for_P.doit(), # important
            self.omega_equations + self.v_equations) # for Ex: omega[1] -> nu1), v[1] -> x', ....
        # subs_connections
        sub_conn_equations_for_P = self.sub_connections(sub_equations_for_P)
        print(sub_conn_equations_for_P)
        # to equate the coefficients
        left_coeffs = poly(sub_conn_equations_for_P.args[0], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        right_coeffs = poly(sub_conn_equations_for_P.args[1], TatarinovSystem.right_part_Eqs(self.omega_equations)).coeffs()
        final_equations_for_P = [Eq(left, right) for left, right in zip(left_coeffs, right_coeffs)]
        self.Ps = final_equations_for_P
        return final_equations_for_P
    
    def create_Q(self, F, r_p):
        self.Q = [F.dot(r_p.diff(x)) for x in self.q]
    
    def Q_dw_by_dv(self, i):
        Q_dw_by_dv = lambda i,a: self.Q[i]*Derivative(self.right_part_Eqs(S.sub_connections_to_list(S.v_equations))[i],
                                                      self.right_part_Eqs(self.omega_equations)[a]).doit()
        Q_dw_by_dv_sum = lambda a: Q_dw_by_dv(0,a) + Q_dw_by_dv(1,a) + Q_dw_by_dv(2,a)
        return Q_dw_by_dv_sum(i)
    
    def tatarinov_equation__depricated(self, i):
        left, right = 0, 0;
        left += diff(diff(L, omega[k]), t)
        left += poisson_bracket(P[k], L)
        right += poisson_bracket(P[k], Sum(omega[k]*P[k], (k, 1, self.N)))
        right += Sum(Q[l]*diff(v[l], omega[k]), (l, 1, self.N))
        Eq(left, right)
        
    def create_bracket_sum(self):
        """ not good """
#         bracket_sum = nu1*(p[1]*cos(alpha) + p[2]*sin(alpha)) + nu2*(-p[1]*sin(alpha) + p[2]*cos(alpha)) + nu3*p[3]
        bracket_sum = [x*y for x,y in zip(S.right_part_Eqs(S.Ps), S.right_part_Eqs(S.omega_equations))]
        res = 0
        for x in range(3):
            res += bracket_sum[x]
        return res
    
    def create_fs(self, fs):
        """ not good """
        return fs.subs({
            nu1: Function('nu1')(t),
            nu2: Function('nu2')(t),
            nu3: Function('nu3')(t)
        })

    def tatarinov_equation(self, i):
        left, right = 0,0
        left = S.sub_connections(self.L).args[1]
        left = left.diff(self.right_part_Eqs(self.omega_equations)[i])
        left += self.poisson_bracket(self.Ps[i].args[1], self.sub_connections(self.L).args[1])
        left = self.create_fs(left).diff(t)
        
        right += self.poisson_bracket(self.Ps[i].args[1], self.create_bracket_sum())
        right += simplify(self.Q_dw_by_dv(i))
        right = self.create_fs(right)
        
        ps = lambda i: self.sub_connections(self.diff_hack(self.L.args[1], self.right_part_Eqs(self.v_equations)[i]))
        
        self.tatarinov_equations[i] = Eq(left, right).subs({p[1]: ps(0), p[2]: ps(1), p[3]: ps(2)})
        self.tatarinov_equations[i] = simplify(self.tatarinov_equations[i])
        
        return self.tatarinov_equations[i]

    def display_tatarinov_equations(self):
        for eq in self.tatarinov_equations:
            display(eq)

In [35]:
L, m, x, y, t, alpha = symbols('L, m, x, y, t, alpha')
nu1, nu2, nu3 = symbols('nu1, nu2, nu3')
W, T = symbols('W, T')
s = symbols('s')
P = IndexedBase('P')
p = IndexedBase('p')
omega = IndexedBase('omega')
v = IndexedBase('v')
Q = IndexedBase('Q')
k = symbols('k', cls=Idx)
J = IndexedBase('J')

xi, eta = symbols('xi, eta')
e = {}
e['x'] = Matrix([1,0])
e['y'] = Matrix([0,1])
e['xi'] = Matrix([cos(alpha), sin(alpha)])
e['eta'] = Matrix([-sin(alpha), cos(alpha)])

In [36]:
A, C, c = symbols('A, C, c')

psi, theta, phi = symbols('psi, theta, phi') # v

g = symbols('g')

### Определим Лагранжиан

In [37]:
L = Eq(L, A*(Derivative(theta,t)**2 + Derivative(psi,t)**2*sin(theta)**2)/2 +
          C*(Derivative(phi, t) + Derivative(psi, t)*cos(theta))**2/2 -
          m*g*cos(theta))
L

Eq(L, A*(sin(theta)**2*Derivative(psi, t)**2 + Derivative(theta, t)**2)/2 + C*(cos(theta)*Derivative(psi, t) + Derivative(phi, t))**2/2 - g*m*cos(theta))

### Определим связи 

In [38]:
conn1 = Eq(nu1, Derivative(psi,t)*sin(theta)*sin(phi) + Derivative(theta,t)*cos(phi))
conn2 = Eq(nu2, Derivative(psi,t)*sin(theta)*cos(phi) - Derivative(theta,t)*sin(phi))
conn3 = Eq(nu3, Derivative(phi,t) + Derivative(psi,t)*cos(theta))

In [39]:
simplify(conn1.args[1]**2 + conn2.args[1]**2)

sin(theta)**2*Derivative(psi, t)**2 + Derivative(theta, t)**2

In [40]:
sol = solve([conn1, conn2, conn3], [Derivative(psi,t), Derivative(phi,t), Derivative(theta,t)])
sol

{Derivative(psi, t): (nu1*sin(phi) + nu2*cos(phi))/sin(theta),
 Derivative(phi, t): -nu1*sin(phi)/tan(theta) - nu2*cos(phi)/tan(theta) + nu3,
 Derivative(theta, t): nu1*cos(phi) - nu2*sin(phi)}

In [41]:
sol.items()

dict_items([(Derivative(psi, t), (nu1*sin(phi) + nu2*cos(phi))/sin(theta)), (Derivative(phi, t), -nu1*sin(phi)/tan(theta) - nu2*cos(phi)/tan(theta) + nu3), (Derivative(theta, t), nu1*cos(phi) - nu2*sin(phi))])

In [42]:
conn1 = Eq(Derivative(phi,t), sol[Derivative(phi,t)])
conn2 = Eq(Derivative(psi,t), sol[Derivative(psi,t)])
conn3 = Eq(Derivative(theta,t), sol[Derivative(theta,t)])

In [43]:
display(conn1)
display(conn2)
display(conn3)

Eq(Derivative(phi, t), -nu1*sin(phi)/tan(theta) - nu2*cos(phi)/tan(theta) + nu3)

Eq(Derivative(psi, t), (nu1*sin(phi) + nu2*cos(phi))/sin(theta))

Eq(Derivative(theta, t), nu1*cos(phi) - nu2*sin(phi))

### Создадим объект для механической системы с уравнениями Татаринова 

In [44]:
S = TatarinovSystem()

### Заполним что нужно (связи, уравнения для $\omega_i$, $v_i$, $F$)

In [45]:
S.set_L(L)

In [46]:
S.set_p([p[1], p[2], p[3]]) # это скорее внутренняя переменная и её не надо задавать
S.set_q([phi, psi, theta])

In [47]:
S.set_connections([conn1, conn2, conn3])
S.connections

[Eq(Derivative(phi, t), -nu1*sin(phi)/tan(theta) - nu2*cos(phi)/tan(theta) + nu3),
 Eq(Derivative(psi, t), (nu1*sin(phi) + nu2*cos(phi))/sin(theta)),
 Eq(Derivative(theta, t), nu1*cos(phi) - nu2*sin(phi))]

In [50]:
S.set_omega_equations(omegas=[omega[1], omega[2], omega[3]],
                                                 equations=[nu1, nu2, nu3])
S.omega_equations

[Eq(omega[1], nu1), Eq(omega[2], nu2), Eq(omega[3], nu3)]

In [51]:
S.set_v_equations(vs=[v[1], v[2], v[3]],
                  equations=[Derivative(phi,t), Derivative(psi,t), Derivative(theta,t)])
S.v_equations

[Eq(v[1], Derivative(phi, t)),
 Eq(v[2], Derivative(psi, t)),
 Eq(v[3], Derivative(theta, t))]

In [52]:
F = Matrix([0,0])
S.set_F(F)
S.F

Matrix([
[0],
[0]])

### Создадим $P_{\alpha}$

In [53]:
Ps = S.create_P()
Ps

Eq(Sum(P[k]*omega[k], (k, 1, 3)), Sum(p[k]*v[k], (k, 1, 3)))
Eq(nu1*P[1] + nu2*P[2] + nu3*P[3], -nu1*sin(phi)*p[1]/tan(theta) + nu1*sin(phi)*p[2]/sin(theta) + nu1*cos(phi)*p[3] - nu2*sin(phi)*p[3] - nu2*cos(phi)*p[1]/tan(theta) + nu2*cos(phi)*p[2]/sin(theta) + nu3*p[1])


[Eq(P[1], -sin(phi)*p[1]/tan(theta) + sin(phi)*p[2]/sin(theta) + cos(phi)*p[3]),
 Eq(P[2], -sin(phi)*p[3] - cos(phi)*p[1]/tan(theta) + cos(phi)*p[2]/sin(theta)),
 Eq(P[3], p[1])]

### Получение $Q_i$

In [54]:
S.create_Q(S.F, S.create_r()['p'])
S.Q

[0, 0, 0]

### $ L^* $

In [55]:
simplify(S.sub_connections(L))

Eq(L, A*nu1**2/2 + A*nu2**2/2 + C*nu3**2/2 - g*m*cos(theta))

### Получаем уравнения

In [56]:
for i in range(3):
    S.tatarinov_equation(i)

In [57]:
S.display_tatarinov_equations()

Eq(A*Derivative(nu1(t), t), A*nu2*nu3(t) - C*nu3*nu2(t))

Eq(A*Derivative(nu2(t), t), -A*nu1*nu3(t) + C*nu3*nu1(t))

Eq(C*Derivative(nu3(t), t), A*(nu1*nu2(t) - nu2*nu1(t)))